This demo is using the following implemented stories:
- RSPY-25
- RSPY-85
- RSPY-100
- RSPY-115

Install the needed libraries

In [1]:
!pip install boto3
!(cd $RSPY_WHL_DIR && pip install rs_server_libraries-*.whl )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 32.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.7 MB/s eta 0:00:00
Processing ./rs_server_libraries-0.0.0.post1.dev0+5fe9de7-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 939.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# We'll use boto3 to monitor the s3 bucket. 
# Note: the S3_ACCESSKEY, S3_SECRETKEY and S3_ENDPOINT are given in the docker-compose.yml file.
import boto3
import os

s3_session = boto3.session.Session()
s3_client = s3_session.client(
    service_name="s3",
    aws_access_key_id=os.environ["S3_ACCESSKEY"],
    aws_secret_access_key=os.environ["S3_SECRETKEY"],
    endpoint_url=os.environ["S3_ENDPOINT"],
    region_name=os.environ["S3_REGION"],
)
bucket_name = "tmp-download"
final_bucket_name = "final-catalog"
bucket_dir = "stations"
bucket_url = f"s3://{bucket_name}/{bucket_dir}"

# If bucket is already created, clear all files in order to start fresh for each demo. 
if bucket_name in [bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]]:
    if 'Contents' in s3_client.list_objects(Bucket=bucket_name):
        objects = s3_client.list_objects(Bucket=bucket_name)['Contents']
        for obj in objects:
            # clear up the bucket
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
else:
    s3_client.create_bucket(Bucket=bucket_name)

# create the final bucket
s3_client.create_bucket(Bucket=final_bucket_name)

print(f"{bucket_name} empty ?: ", 'Contents' not in s3_client.list_objects(Bucket=bucket_name))
print(f"{final_bucket_name} empty ?: ", 'Contents' not in s3_client.list_objects(Bucket=final_bucket_name))

tmp-download empty ?:  True
final-catalog empty ?:  True


A bucket "tmp-download" is created for the purpose of this demo. Thus, the cadip and adgs prefect flows will be asking for the rs-server endpoints to download the files from CADIP and ADGS stations and to upload them to "s3://tmp-download/stations/<station_name>"
After a succesfull upload to s3 bucket, the update stac catalog endpoint is called to update the catalog and to copy the file to the "final-catalog" bucket.

In [3]:
from dataclasses import dataclass
import requests

@dataclass
class Collection:
    """A collection for test purpose."""

    user: str
    name: str

    @property
    def id_(self) -> str:
        """Returns the id."""
        return f"{self.user}_{self.name}"

    @property
    def properties(self):
        """Returns the properties."""
        return {
            "id": self.name,
            "type": "Collection",
            "links": [
                {
                    "rel": "items",
                    "type": "application/geo+json",
                    "href": f"http://localhost:8082/collections/{self.name}/items",
                },
                {"rel": "parent", "type": "application/json", "href": "http://localhost:8082/"},
                {"rel": "root", "type": "application/json", "href": "http://localhost:8082/"},
                {
                    "rel": "self",
                    "type": "application/json",
                    "href": f"""http://localhost:8082/collections/{self.name}""",
                },
                {
                    "rel": "license",
                    "href": "https://creativecommons.org/licenses/publicdomain/",
                    "title": "public domain",
                },
            ],
            "extent": {
                "spatial": {"bbox": [[-94.6911621, 37.0332547, -94.402771, 37.1077651]]},
                "temporal": {"interval": [["2000-02-01T00:00:00Z", "2000-02-12T00:00:00Z"]]},
            },
            "license": "public-domain",
            "description": "Some description",
            "stac_version": "1.0.0",
        }
    
user = "DemoUser"
mission = "s1"
url_catalog = "http://rs-server-catalog:8000"

# Create the collection for DemoUser
collection_type = Collection(user, f"{mission}_aux")
response = requests.post(url_catalog + f"/catalog/{user}/collections", json=collection_type.properties)

In [5]:


from datetime import datetime

from rs_workflows.common import (
    PrefectFlowConfig,
    download_flow,
)
    
def run_flow(user, url, url_catalog, station, mission, tmp_local_download, bucket_url, no_of_tasks, start_date, stop_date):
    # start the prefect flow
    download_flow(PrefectFlowConfig(user,
                                    url,
                                    url_catalog,
                                    station,
                                    mission,
                                    tmp_local_download,
                                    bucket_url,
                                    no_of_tasks,
                                    datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%SZ"),
                                    datetime.strptime(stop_date, "%Y-%m-%dT%H:%M:%SZ"),                                   
            )
)


stations = ["CADIP", "ADGS"]
url = "http://rs-server-{}:8000"

tmp_local_download = "/tmp/{}_tmp"
no_of_tasks = 1



for station in stations:
    run_flow(user,
             url.format(station.lower()),
             url_catalog,
             station,
             mission,
             tmp_local_download.format(station),
             bucket_url + f"/{station}",
             no_of_tasks,
             "2014-01-01T12:00:00Z",
             "2024-02-20T12:00:00Z",
             )    
    

22:12:13.268 | INFO    | prefect.engine - Created flow run 'divergent-heron' for flow 'download-flow'

22:12:13.270 | INFO    | Flow run 'divergent-heron' - View at http://prefect-server:4200/flow-runs/flow-run/43c9430a-15b3-44c7-999d-6ca6939d5f66

22:12:13.271 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`

22:12:13.273 | INFO    | distributed.scheduler - State start

22:12:13.278 | INFO    | distributed.scheduler -   Scheduler at:     tcp://127.0.0.1:37225

...

22:12:23.450 | INFO    | distributed.scheduler - Remove worker <WorkerState 'tcp://127.0.0.1:36227', name: 3, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1710195143.4502816')

22:12:23.451 | INFO    | distributed.scheduler - Lost all workers

22:12:24.041 | INFO    | distributed.scheduler - Scheduler closing due to unknown reason...

22:12:24.042 | INFO    | distributed.scheduler - Scheduler closing all comms

22:12:24.044 | INFO    | Flow run 'vigilant-lori' - Finished in state Completed()